In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import os

In [7]:
def train_random_forest_model(df, include_ratios=False, cv_folds=5):
    """
    Trains a Random Forest model on censored=0 data to predict final_expandEvts and validates on all data
    to assess generalizability. Uses relative error metrics (MAPE, MdAPE, Normalized RMSE) and cross-validation.

    Parameters:
    - df (pandas.DataFrame): Preprocessed DataFrame with solver features, ratio features, and final_expandEvts.
    - include_ratios (bool): If True, includes ratio features; if False, excludes them (default: False).
    - save_plots (bool): If True modelu, saves scatter and feature importance plots to output_dir.
    - output_dir (str): Directory to save plots and results (default: 'plots').
    - cv_folds (int): Number of cross-validation folds (default: 5).

    Returns:
    - dict: Contains relative error metrics, CV scores, feature importance, and predictions for censored=0 test set and all data.
    """
    try:
        # Validate input
        if not isinstance(df, pd.DataFrame):
            raise TypeError(f"Input 'df' must be a pandas DataFrame, got {type(df)}")
        print(f"Input DataFrame type: {type(df)}")
        # print(f"Input columns: {df.columns.tolist()}")
        
        # Feature selection
        exclude_cols = ['filename', 'final_expandEvts', 'stop_iter', 'final_maxStackDepth']
        if not include_ratios:
            exclude_cols.extend([f'expandEvts_ratio_{i}' for i in range(1, 6)])
            exclude_cols.extend([f'pruneBacktrackEvts_ratio_{i}' for i in range(1, 6)])
        features = [col for col in df.columns if col not in exclude_cols]
        # print(f"Selected features ({len(features)}): {features}")
        
        # Print target statistics
        print("\nTarget Statistics (final_expandEvts):")
        mean_censored = df[df['censored'] == 0]['final_expandEvts'].mean()
        std_censored = df[df['censored'] == 0]['final_expandEvts'].std()
        mean_all = df['final_expandEvts'].mean()
        std_all = df['final_expandEvts'].std()
        print(f"Mean (censored=0): {mean_censored:.2f}")
        print(f"Std Dev (censored=0): {std_censored:.2f}")
        print(f"Mean (all data): {mean_all:.2f}")
        print(f"Std Dev (all data): {std_all:.2f}")
        
        # Calculate relative error metrics
        def calculate_mape(y_true, y_pred):
            mask = y_true != 0
            return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if mask.sum() > 0 else np.nan
        
        def calculate_mdape(y_true, y_pred):
            mask = y_true != 0
            return np.median(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if mask.sum() > 0 else np.nan
        
        def calculate_normalized_rmse(y_true, y_pred, mean_y):
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            return rmse / mean_y if mean_y != 0 else np.nan
        
        # Train on censored=0 data
        print("\nTraining Random Forest on censored=0 data...")
        df_censored = df[df['censored'] == 0]
        if df_censored.empty:
            print("Warning: No censored=0 instances found. Exiting.")
            return None
        
        X_censored = df_censored[features]
        y_censored = df_censored['final_expandEvts']
        X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_censored, y_censored, test_size=0.2, random_state=42)
        
        rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_model.fit(X_train_c, y_train_c)
        
        # Evaluate on censored=0 test set
        y_pred_c = rf_model.predict(X_test_c)
        rmse_censored = np.sqrt(mean_squared_error(y_test_c, y_pred_c))
        norm_rmse_censored = calculate_normalized_rmse(y_test_c, y_pred_c, mean_censored)
        mape_censored = calculate_mape(y_test_c, y_pred_c)
        mdape_censored = calculate_mdape(y_test_c, y_pred_c)
        cv_scores = cross_val_score(rf_model, X_censored, y_censored, cv=cv_folds, scoring='neg_root_mean_squared_error')
        cv_norm_rmse_censored = (-cv_scores.mean()) / mean_censored if mean_censored != 0 else np.nan
        print(f"Censored=0 Test Set RMSE: {rmse_censored:.4f}")
        print(f"Censored=0 Test Set Normalized RMSE: {norm_rmse_censored:.4%}")
        print(f"Censored=0 Test Set MAPE: {mape_censored:.2f}%")
        print(f"Censored=0 Test Set MdAPE: {mdape_censored:.2f}%")
        print(f"Censored=0 CV Normalized RMSE: {cv_norm_rmse_censored:.4%}")
        
        importance_censored = pd.DataFrame({
            'feature': X_censored.columns,
            'importance': rf_model.feature_importances_
        }).sort_values('importance', ascending=False)
        print("\nCensored=0 Feature Importance (Top 10):")
        print(importance_censored.head(10))
        
        # Validate on all data
        print("\nValidating on all data...")
        X_all = df[features]
        y_all = df['final_expandEvts']
        y_pred_all = rf_model.predict(X_all)
        rmse_all = np.sqrt(mean_squared_error(y_all, y_pred_all))
        norm_rmse_all = calculate_normalized_rmse(y_all, y_pred_all, mean_all)
        mape_all = calculate_mape(y_all, y_pred_all)
        mdape_all = calculate_mdape(y_all, y_pred_all)
        print(f"All Data RMSE: {rmse_all:.4f}")
        print(f"All Data Normalized RMSE: {norm_rmse_all:.4%}")
        print(f"All Data MAPE: {mape_all:.2f}%")
        print(f"All Data MdAPE: {mdape_all:.2f}%")

        # Add cross-validation for all data
        print("\nPerforming cross-validation on all data...")
        cv_scores_all = cross_val_score(rf_model, X_all, y_all, cv=cv_folds, scoring='neg_root_mean_squared_error')
        cv_norm_rmse_all = (-cv_scores_all.mean()) / mean_all if mean_all != 0 else np.nan
        print(f"All Data CV Normalized RMSE: {cv_norm_rmse_all:.4%}")
        
        return {
            'rmse_censored_test': rmse_censored,
            'norm_rmse_censored_test': norm_rmse_censored,
            'mape_censored_test': mape_censored,
            'mdape_censored_test': mdape_censored,
            'cv_norm_rmse_censored': cv_norm_rmse_censored,
            'rmse_all': rmse_all,
            'norm_rmse_all': norm_rmse_all,
            'mape_all': mape_all,
            'mdape_all': mdape_all,
            'feature_importance': importance_censored,
        }
    
    except Exception as e:
        print(f"Error in train_random_forest_model: {e}")
        return None

df = pd.read_excel("structured_data.xlsx")
results = train_random_forest_model(df, include_ratios=True)
results

Input DataFrame type: <class 'pandas.core.frame.DataFrame'>

Target Statistics (final_expandEvts):
Mean (censored=0): 53227800.02
Std Dev (censored=0): 174281183.67
Mean (all data): 761155886.29
Std Dev (all data): 636531467.58

Training Random Forest on censored=0 data...
Censored=0 Test Set RMSE: 62180933.2255
Censored=0 Test Set Normalized RMSE: 116.8204%
Censored=0 Test Set MAPE: 21.97%
Censored=0 Test Set MdAPE: 17.91%
Censored=0 CV Normalized RMSE: 137.0923%

Censored=0 Feature Importance (Top 10):
                      feature  importance
24     avg_pruneBacktrackEvts    0.284095
22             avg_expandEvts    0.167974
25     max_pruneBacktrackEvts    0.122997
20                   avg_evts    0.114084
21                   max_evts    0.069977
23             max_expandEvts    0.040398
64  diff_pruneBacktrackEvts_3    0.028627
15       pruneBacktrackEvts_3    0.023523
61  diff_pruneBacktrackEvts_2    0.018367
14               expandEvts_3    0.015499

Validating on all data...
A

{'rmse_censored_test': 62180933.22554364,
 'norm_rmse_censored_test': 1.1682040812920012,
 'mape_censored_test': 21.965653756343215,
 'mdape_censored_test': 17.914652333312624,
 'cv_norm_rmse_censored': 1.3709234063884577,
 'rmse_all': 274590148.7097964,
 'norm_rmse_all': 0.36075415516682263,
 'mape_all': 19.77972990133863,
 'mdape_all': 20.143298216034644,
 'feature_importance':                        feature  importance
 24      avg_pruneBacktrackEvts    0.284095
 22              avg_expandEvts    0.167974
 25      max_pruneBacktrackEvts    0.122997
 20                    avg_evts    0.114084
 21                    max_evts    0.069977
 ..                         ...         ...
 4              backtrackEvts_1    0.000000
 3         pruneBacktrackEvts_1    0.000000
 50  pruneBacktrackEvts_ratio_1    0.000000
 2                 expandEvts_1    0.000000
 19                    censored    0.000000
 
 [71 rows x 2 columns]}